In [86]:
import cv2
import faiss
import os
import numpy as np
import pandas as pd
import scipy
from scipy.spatial.distance import cosine
import random
from sklearn.metrics import classification_report, accuracy_score
%matplotlib inline

In [94]:
from util import *

In [100]:
TMP_DIR = 'tmp'
FRAMES_DIR = 'tmp/frames'
TRAIN_VIDEOS_DIR = 'data/train_videos'
TEST_VIDEOS_DIR = 'data/test_videos'
FEATURE_EXTRACTOR = color_histogram_features

In [101]:
fpaths = [os.path.join(TRAIN_VIDEOS_DIR, fpath) for fpath in os.listdir(TRAIN_VIDEOS_DIR)]
train_df = get_dataframe(fpaths,
                         feature_extractor=FEATURE_EXTRACTOR,
                         #preprocessor=preprocess_image_load,
                         #write_frames_dir=FRAMES_DIR,
                         mod_filter=15,
                         )
train_df.head()

Done 0.5
Done 1.0


,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_719,x_720,x_721,x_722,x_723,x_724,x_725,x_726,x_727,x_728
0,data/train_videos/1947 - Fun and Fancy Free.avi,13.4,0.00002,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.008539,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,data/train_videos/1947 - Fun and Fancy Free.avi,52.9,0.00000,0.0,0.000000,0.000000,0.000767,0.000373,0.001060,0.000838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,data/train_videos/1947 - Fun and Fancy Free.avi,170.5,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.001484,0.012173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,data/train_videos/1947 - Fun and Fancy Free.avi,232.0,0.00002,0.0,0.004461,0.037771,0.051427,0.025042,0.012748,0.012890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,data/train_videos/1947 - Fun and Fancy Free.avi,300.6,0.00003,0.0,0.000000,0.000252,0.001514,0.003169,0.002614,0.007459,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
frame_times = train_df['frame_time']
labels = train_df.video_path.values
frame_vectors = train_df.drop(['frame_time', 'video_path'], axis=1).values
frame_vectors

array([[2.01873390e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 2.43739734e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.40260647e-04, 1.60996511e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.08400045e-03, 3.30972904e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [107]:
clf = FaissVideoClassifier(frame_vectors, labels, treshold=0.7)

In [108]:
test_videos = [os.path.join(TEST_VIDEOS_DIR, f) for f in os.listdir(TEST_VIDEOS_DIR)]
for fpath in test_videos:
    test_df = get_dataframe([fpath],
                            feature_extractor=FEATURE_EXTRACTOR,
                            #preprocessor=preprocess_image_restore,
                            write_frames_dir=FRAMES_DIR,
                            ssim_threshold=None)   
    test_df_frame_vectors = test_df.drop(['frame_time', 'video_path'], axis=1).values
    predicted_label, votes, dists, indices = clf.classify(test_df_frame_vectors)
    print(fpath)
    print('Classified as', predicted_label)
    #print(votes, dists, indices)
    print(dists)
    print('Closest frames')
    for v, t in zip(votes, frame_times[indices]):
        print(v, t)
    print()

Done 1.0
data/test_videos/the_fun_and_fancy_tree_6.mp4
Classified as data/train_videos/1947 - Fun and Fancy Free.avi
[0.77374136 0.7589376  0.76052034 0.8097659  0.7706897  0.7119568
 0.6817409  0.70732677 0.75944746 0.711558   0.7005333  0.70391834]
Closest frames
data/train_videos/1947 - Fun and Fancy Free.avi 3221.7
data/train_videos/1947 - Fun and Fancy Free.avi 3221.7
data/train_videos/1947 - Fun and Fancy Free.avi 3221.7
data/train_videos/1947 - Fun and Fancy Free.avi 1260.7
data/train_videos/1947 - Fun and Fancy Free.avi 1260.7
data/train_videos/1947 - Fun and Fancy Free.avi 1260.7
miss 1260.7
data/train_videos/1947 - Fun and Fancy Free.avi 2067.3
data/train_videos/1947 - Fun and Fancy Free.avi 1260.7
data/train_videos/1947 - Fun and Fancy Free.avi 1260.7
data/train_videos/1947 - Fun and Fancy Free.avi 3221.7
data/train_videos/1947 - Fun and Fancy Free.avi 4007.8

Done 1.0
data/test_videos/the_fun_and_fancy_tree_4.mp4
Classified as data/train_videos/1947 - Fun and Fancy Free.avi